Assignment:
Now that you've learned about random forests and decision trees let's do an exercise in accuracy. You know that random forests are basically a collection of decision trees. But how do the accuracies of the two models compare?

So here's what you should do. Pick a dataset. It could be one you've worked with before or it could be a new one. Then build the best decision tree you can.

Now try to match that with the simplest random forest you can. For our purposes measure simplicity with runtime. Compare that to the runtime of the decision tree. This is imperfect but just go with it.

Hopefully out of this you'll see the power of random forests, but also their potential costs. Remember, in the real world you won't necessarily be dealing with thousands of rows. It could be millions, billions, or even more.



In [1]:
import numpy as np
import pandas as pd
import matplotlib.pylab as plt
import seaborn as sns
from sklearn import neighbors
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RandomizedSearchCV
from sklearn import tree
from IPython.display import Image  # A convenience for displaying visualizations.
import pydotplus  # Package for rendering our tree.
import graphviz  # Package for rendering our tree.

In [2]:
#try random forest on ramen rating dataset that linear regression and KNN didn't suceed with
# %load ../20/20.5_knn_challenge_drill.py
#!/usr/bin/env python

In [3]:
# %load ../../utility/version_print.py
#record module versions used in cell 1
#
def version_recorder():
    '''
    only works if import is first cell run. prints and then returns dictionary with modules:version.
    '''
    import pkg_resources
    resources = In[1].splitlines()
    ##ADD: drop lines if not _from_ or _import_
    version_dict = { resource.split()[1].split(".")[0] : pkg_resources.get_distribution(resource.split()[1].split(".")[0]).version for resource in resources }
    return version_dict
version_recorder()

{'numpy': '1.16.4',
 'pandas': '0.25.0',
 'matplotlib': '3.1.1',
 'seaborn': '0.9.0',
 'sklearn': '0.0',
 'IPython': '7.6.1',
 'pydotplus': '2.0.2',
 'graphviz': '0.11.1'}

In [4]:
df = pd.read_csv("../../data_sets/ramen-ratings.csv", )
#only include if brand is commonly tested
brand_list = (df.Brand.value_counts() > 10).index
df = df[df["Brand"].isin(brand_list)]
 
#top ten would be separate target
df.drop(columns="Top Ten", inplace=True)

df.dropna(how='any', inplace=True)
df = df[df.Stars.str.isdigit()]

y = df.Stars
y = pd.to_numeric(y)
X = pd.get_dummies(df[["Style", "Country", "Brand"]], drop_first=True)

regression_model = LinearRegression()
regression_model.fit(X, y)
print("rsquare is {}".format(regression_model.score(X, y)))

rsquare is 0.5745301123670111


In [5]:
cross_val_score(regression_model, X, y, cv=5)
# Very poor CV. will try another Regression model.
ridge_model = Ridge()
ridge_model.fit(X, y)
print("rsquare is {}".format(ridge_model.score(X, y)))
print(cross_val_score(ridge_model, X, y, cv=5))

rsquare is 0.5106085635454151
[ 0.01804503  0.14490214  0.12029419  0.0828164  -0.27015293]


In [6]:
knn_model = neighbors.KNeighborsRegressor(n_neighbors=2)
knn_model.fit(X, y)
print("rsquared is {}".format(knn_model.score(X, y)))
print(cross_val_score(knn_model, X, y, cv=5))

rsquared is 0.461693812021445
[-0.78304577 -0.21570234  0.01037006 -0.01967306 -0.58637207]


In [7]:
knn_model2 = neighbors.KNeighborsRegressor(n_neighbors=10, weights='distance')
knn_model2.fit(X, y)
print("rsquared is {}".format(knn_model2.score(X, y)))
print(cross_val_score(knn_model2, X, y, cv=5))

rsquared is 0.6625044221755905
[-0.18861091  0.01602315  0.12318203  0.02487342 -0.47738882]


## Decision Tree Model

In [8]:
tree_model_d5 = tree.DecisionTreeClassifier(criterion='entropy',
                                        max_depth=5)
tree_model_d5.fit(X, y)
print(f"tree score is {tree_model_d5.score(X, y)}")
cv_d5 = cross_val_score(tree_model_d5, X, y, cv=4)
print("cross val scores are {}, avg {}".format(cv_d5, cv_d5.mean()))

tree score is 0.46698564593301434
cross val scores are [0.41064639 0.44656489 0.38846154 0.40384615], avg 0.4123797414091438


In [9]:
tree_model_g = tree.DecisionTreeClassifier(criterion='gini',
                                        max_depth=10)
tree_model_g.fit(X, y)
print(f"tree score is {tree_model_g.score(X, y)}")
cv_g = cross_val_score(tree_model_g, X, y, cv=4)
print("cross val scores are {}, avg {}".format(cv_g, cv_g.mean()))

tree score is 0.5473684210526316
cross val scores are [0.39923954 0.45801527 0.38846154 0.36923077], avg 0.403736779648529


In [10]:
tree_model = tree.DecisionTreeClassifier(criterion='entropy',
                                        max_depth=10)
tree_model.fit(X, y)
print(f"tree score is {tree_model.score(X, y)}")
cv = cross_val_score(tree_model, X, y, cv=4)
print("cross val scores are {}, avg {}".format(cv, cv.mean()))

tree score is 0.5358851674641149
cross val scores are [0.40684411 0.39312977 0.40384615 0.36538462], avg 0.3923011616717535


quite a bit of variation, but much better than the other models!

## Random Forest Model

In [11]:
rfc_mod = RandomForestClassifier(n_estimators=100,
                                criterion='entropy',
                                )
rfc_mod.fit(X, y)
print(f"forest score is {rfc_mod.score(X, y)}")
cv_rfc = cross_val_score(rfc_mod, X, y, cv=4)
print("cross val scores are {}, avg {}".format(cv_rfc, cv_rfc.mean()))

forest score is 0.707177033492823
cross val scores are [0.39543726 0.40076336 0.32307692 0.32692308], avg 0.3615501552840101


In [12]:
rfc_mod_gini = RandomForestClassifier(n_estimators=100,
                                criterion='gini',
                                )
rfc_mod_gini.fit(X, y)
print(f"forest score is {rfc_mod_gini.score(X, y)}")
cv_rfc_gini = cross_val_score(rfc_mod_gini, X, y, cv=4)
print("cross val scores are {}, avg {}".format(cv_rfc_gini, cv_rfc_gini.mean()))

forest score is 0.707177033492823
cross val scores are [0.42585551 0.38931298 0.30769231 0.31153846], avg 0.35859981490949766


## Try random search on forest hyperparameters

In [13]:
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(5, 50, num = 5)]
max_depth.append(None)
# Minimum number of samples for a node split
min_samples_split = [2, 5, None]
# Minimum samples at each leaf
min_samples_leaf = [1, 2, 4]
# Create the random grid
random_grid = {'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf}
rfc_instance = RandomForestClassifier

In [14]:
rfc_random = RandomizedSearchCV(estimator=rfc_instance, n_iter=100, cv=4, param_distributions=random_grid)
rfc_random.fit(X, y)

TypeError: Cannot clone object '<class 'sklearn.ensemble.forest.RandomForestClassifier'>' (type <class 'abc.ABCMeta'>): it does not seem to be a scikit-learn estimator as it does not implement a 'get_params' methods.

In [ ]:
print(f"best random forest parameters are: {rfc_random.best_params_}")
print(f"best estimator is {rfc_random.best_estimator_}")